
# Atividade Named-Entity Recognition (NER)

### Desafio

Crie a rotina para processamento dos textos, preparando os documentos para respeitar a estrutura proposta para rodar uma rede neural capaz de realizar a atividade de reconhecimento de entidades.

Lembre que o resultado deverá ter três colunas (identificador, palavra e tag) como exemplificado na tabela abaixo:

| identificador | palavra | tag |
|:--:|:--:|:--:|
|102|tronco|O|
|102|da|O|
|102|coronaria|vaso|
|102|esquerda|vaso|
|102|com|O|
|102|trajeto|O|

Você deverá considerar as seguintes classes e termos por classe:

-   **vaso**:
	- descendente anterior;
	- coronaria direita;
	- coronaria esquerda;
	- circunflexa;
	- primeiro ramo marginal;
	- segundo ramo marginal;
	- terceiro ramo marginal;
	- primeiro ramo diagonal;
	- segundo ramo diagonal;
	- terceiro ramo diagonal;
	- ventricular posterior;
	- arteria diagonalis;
	- descendente posteiror.
-   **trajeto**:
	- trajeto intramiocardico;
	- origem;
	- retroaortico;
	- interaortopulmonar;
	- valsalva;
	- seio;
	- sinotubular.
-   **placa**:
	- placa;
	- placas;
	- ateromatose.
-   **composição**:
	- calcificada;
	- calcificadas;
	- densamente calcificada;
	- densamente calcificadas;
	- densa;
	- densamente calcificado;
	- parcialmente calcificada;
	- predominantemente calcificada;
	- predominantemente calcificadas;
	- mista;
	- mistas;
	- predominantemente nao calcificada;
	- predominantemente não calcificada;
	- parcialmente não calcificada;
	- principalmente calcificada;
	- principalmente não calcificada;
	- principalmente nao calcificada;
	- predominio calcificado;
	- predominio nao calcificado;
	- não calcificada.
-   **grau**:
	- sem redução luminal;
	- discreta;
	- irregularidades parietais;
	- irregularidade parietal;
	- menor que 50%;
	- irregularidades luminais;
	- irregularidade luminal;
	- irregularidade;
	- irregularidades;
	- proxima de 50%;
	- proximo de 50%;
	- entre 50 e 70%;
	- 50%;
	- acima de 50%;
	- maior que 50%;
	- ao redor de 50%;
	- cerca de 50%;
	- em torno de 50%;
	- acima de 70%;
	- 70%;
	- cerca de 70%;
	- ao redor de 70%;
	- em torno de 70%;
	- moderada;
	- moderada reducao luminal;
	- reducao luminal moderada;
	- estenose moderada;
	- grau pelo menos moderado;
	- grau moderado;
	- acentuada;
	- suboclusao;
	- reducao luminal critica;
	- estenose critica;
	- oclusao;
	- ocluido;
	- ocluida.
-   **modificador V**:
	- modificador V;
	- vulnerabilidade;
	- remodelamento positivo;
	- baixa atenuação;
	- napking ring;
	- anel de guardanapo;
	- spot calcifications;
	- remodelamento arterial positivo.
-   **stent**:
	- stent.
-   **redução stent**:
	- hiperplasia neointimal;
	- neointimal;
	- proliferação neointimal.
-   **enxerto**:
	- enxerto.

In [1]:
# Resolução
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_rows', None)

df = pd.read_excel('https://github.com/pgiaeinstein/ner-aula/raw/master/data_coronaria.xlsx')

In [2]:
df.shape

(439, 1)

In [3]:
df.head()

,texto
0,paciente com escore de calcio de zero fase com...
1,paciente com escore de calcio de zero fase com...
2,paciente com escore de calcio de zero fase com...
3,paciente com escore de calcio zero fase com co...
4,paciente com escore de calcio de zero fase com...


In [4]:
reg_vaso = re.compile(r"(descendente anterior|descendente posterior|coronaria (esquerda|direita)|circunflexa|ventricular posterior|arteria diagonalis|((primeiro|segundo|terceiro|quarto)(\se\s|\s))+(ramos?)?\s?(diagon(al|ais)?|margin(al|ais)?)?)")
reg_trajeto = re.compile(r"(trajeto intramiocardico|retroaórtico|interaortopulmonar|valssalva|sinotubular|seio)")
reg_placa = re.compile(r"(placas?|ateromatose)")
reg_desc_placa = re.compile(r"((predominantemente nao |nao |parcialmente |parcialmente nao |predominantemente |principalmente |principalmente nao |densamente )?calcificadas?|mistas?|predominio calcificado|predominio nao calcificado|densamente calcificado|densa)")
reg_oclusao = re.compile(r"(oclusao|suboclusao|acentuada|\b70%?\b|\b50%?\b|moderada\s?(reducao)?\s?(luminal)?|discreta\b|(irregularidades?\s?(pariet(ais|al)|lumin(al|ais))?)|menor que 50%?|maior que 50%?|entre 50%? e 70%?|acima de 50%?|ao redor de 50%?|cerca de 50%?|em torno de 50%?|acima de 70%?|cerca de 70%?|ao redor de 70%?|em torno de 70%?|reducao luminal critica|estenose critica|ocluid(o|a)|reducao luminal moderada|estenose moderada|grau pelo menos moderado|grau moderado|proxima de 50%?|proximo de 50%?)")
reg_modificador = re.compile(r"(modificador v|vulnerabilidade|remodelamento positivo|baixa atenuacao|napking ring|anel de guardanapo|spot calcifications|baixa atenuacao|remodelamento arterial positivo)")
reg_stent = re.compile(r"(stent)")
reg_red_stent = re.compile(r"(proliferacao neointimal|hiperplasia neointimal|hiperplasia|neointimal)")
reg_enxerto = re.compile(r"(enxerto)")

In [5]:
lista_documentos = df['texto'].tolist()

In [6]:
def encontra_tags(texto):
    
    lista_retorno = list()
    
    for item in reg_vaso.finditer(texto):
        lista_retorno.append({
            'classe'  : 'vaso',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
    
    for item in reg_trajeto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'trajeto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_desc_placa.finditer(texto):
        lista_retorno.append({
            'classe'  : 'desc_placa',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_oclusao.finditer(texto):
        lista_retorno.append({
            'classe'  : 'oclusao',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_modificador.finditer(texto):
        lista_retorno.append({
            'classe'  : 'modificador',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_red_stent.finditer(texto):
        lista_retorno.append({
            'classe'  : 'red_stent',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })
        
    for item in reg_enxerto.finditer(texto):
        lista_retorno.append({
            'classe'  : 'enxerto',
            'termo'   : item.group(),
            'inicial' : item.start(),
            'final'   : item.end()
        })

    return sorted(lista_retorno, key = lambda x : x['inicial'])

In [7]:
def parse_texto(texto, lista_tags, indice):
    
    lista_retorno = list()
    lista_tmp = list()
    
    tamanho_texto = len(texto)
    tamanho_lista = len(lista_tags)
    
    ultima_pos = 0
    
    for index, achado in enumerate(lista_tags):
        
        if achado['inicial'] != 0 and index == 0:
            lista_tmp.append((texto[0:achado['inicial']], 'O'))
            ultima_pos = achado['inicial']

        if ultima_pos != achado['inicial']:
            lista_tmp.append((texto[ultima_pos:achado['inicial']], 'O'))
            
        lista_tmp.append((texto[achado['inicial']:achado['final']], achado['classe']))
        ultima_pos = achado['final']
            
        if index + 1 == tamanho_lista:
            lista_tmp.append((texto[ultima_pos:], 'O'))
        
    for item in lista_tmp:
        tag_atual = item[1]
        for palavra in item[0].split(' '):
            if palavra == '':
                continue
            lista_retorno.append({
                'identificador' : indice,
                'palavra'       : palavra,
                'tag'           : tag_atual
            }) 
    
    return lista_retorno
        

In [8]:
index = 1
lista_resultante = list()

for documento in lista_documentos:
    lista_tags_documento = encontra_tags(documento)
    lista_resultante.extend(parse_texto(documento, lista_tags_documento, index))
    index += 1

In [9]:
df_para_nn = pd.DataFrame(lista_resultante)

In [10]:
df_para_nn.head(200)

,identificador,palavra,tag
0,1,paciente,O
1,1,com,O
2,1,escore,O
3,1,de,O
4,1,calcio,O
5,1,de,O
6,1,zero,O
7,1,fase,O
8,1,com,O
9,1,contraste,O


### Modelos

In [11]:
import pandas as pd
import numpy as np

# data = pd.read_excel("new_output.xlsx")

In [12]:
data = df_para_nn

In [13]:
# data = data.fillna(method="ffill")

In [14]:
data.tail(10)

,identificador,palavra,tag
48523,439,do,O
48524,439,mesmo,O
48525,439,lado,O
48526,439,compativel,O
48527,439,com,O
48528,439,sequela,O
48529,439,de,O
48530,439,complexo,O
48531,439,granulomatoso,O
48532,439,primario,O


In [15]:
f'Total de documentos na base: {len(data.identificador.unique())}'

'Total de documentos na base: 439'

In [16]:
words = list(set(data["palavra"].values))
words.append("ENDPAD")
n_words = len(words)
n_words

638

In [17]:
tags = list(set(data["tag"].values))
n_tags = len(tags)
n_tags

7

In [18]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["palavra"].values.tolist(),
                                                     s["tag"].values.tolist())]
        self.grouped = self.data.groupby("identificador").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["identificador".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [19]:
getter = SentenceGetter(data)

In [20]:
sent = getter.get_next()

In [21]:
sentences = getter.sentences

In [22]:
max_len = 300
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [24]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [25]:
word2idx['ENDPAD']

637

In [26]:
word2idx

{'326': 0,
 'esquerdo': 1,
 'bifurcada': 2,
 'oclusao': 3,
 'hilares': 4,
 'na': 5,
 'subrramos': 6,
 '19': 7,
 'ambos': 8,
 'granulomatoso': 9,
 'sexo': 10,
 '163': 11,
 'curta': 12,
 'fino': 13,
 'ramificado': 14,
 'excentrica': 15,
 '559': 16,
 'irriga': 17,
 '167': 18,
 'parede': 19,
 'recesso': 20,
 'parietal': 21,
 'porem': 22,
 'ligeiramente': 23,
 'tres': 24,
 'todos': 25,
 'istmo': 26,
 'opacidade': 27,
 'sistolicas': 28,
 'direitas': 29,
 'contraste': 30,
 'luminais': 31,
 'logo': 32,
 'coronariano': 33,
 'pulmonares': 34,
 'atingindo': 35,
 'coronariana': 36,
 'predominando': 37,
 'inerentes': 38,
 'as': 39,
 'das': 40,
 'grande': 41,
 'dois': 42,
 'ateroscleroticas': 43,
 'particularidades': 44,
 '275': 45,
 'movimento': 46,
 'cruzando': 47,
 'ramificados': 48,
 'ventricular': 49,
 'artefatos': 50,
 'apenas': 51,
 '200': 52,
 '151': 53,
 'hiato': 54,
 'calcificadas': 55,
 'lesoes': 56,
 'identificam': 57,
 'micronodulo': 58,
 'parcialmente': 59,
 'determinando': 60,
 'perce

In [27]:
X[0]

array([ 97, 511, 388, 250, 491, 250, 123, 370, 511,  30, 497, 536, 587,
       472,  90, 497, 536, 511, 215, 140, 564, 570, 621, 343, 529, 191,
       140, 569, 492, 529, 191, 250,  41, 308, 511, 215, 140, 564, 570,
       553, 372, 140, 270, 361, 301, 250,  13, 564, 440, 574, 610, 492,
       569, 511, 215, 140, 564, 570, 553, 548, 190, 250,  41, 564, 440,
       574, 610, 372, 548, 190, 250, 580, 564, 440, 574, 610, 194, 529,
       556, 140,  49, 556, 511, 215, 140, 564, 570, 497, 494, 246,  51,
       443,  71, 242, 511, 215, 140, 564, 570, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637,
       637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 637, 63

In [28]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [29]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [30]:
lista_documentos[1]

'paciente com escore de calcio de zero fase com contraste coronaria direita dominante tronco da coronaria esquerda com trajeto e calibre normais bifurcando em descendente anterior e circunflexa arteria descendente anterior com trajeto e calibre normais primeiro e segundo ramos diagonais de pequeno calibre ambos com discretas irregularidades parietais proximais arteria circunflexa com trajeto e calibre normais primeiro ramo marginal de pequeno calibre sem reducao luminal segundo ramo marginal de grande calibre sem reducao luminal coronaria direita com trajeto e calibre normais arterias descendente posterior e ventricular posterior com trajeto e calibre normais'

In [31]:
y[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0,
       0, 0, 6, 6, 0, 6, 0, 6, 6, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 0, 0, 0,
       0, 0, 0, 5, 5, 0, 0, 6, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0,
       6, 6, 6, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 6, 6, 0, 6, 6,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d

In [32]:
tag2idx

{'O': 0,
 'desc_placa': 1,
 'placa': 2,
 'trajeto': 3,
 'modificador': 4,
 'oclusao': 5,
 'vaso': 6}

In [33]:
from tensorflow.keras.utils import to_categorical

In [34]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=6)

### BI LSTM

In [37]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [38]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=32, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

In [39]:
model = Model(input, out)

In [40]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [41]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

Train on 355 samples, validate on 40 samples
Epoch 1/15
355/355 [==============================] - 5s 15ms/sample - loss: 0.9709 - accuracy: 0.8744 - val_loss: 0.3864 - val_accuracy: 0.8999
Epoch 2/15
355/355 [==============================] - 4s 10ms/sample - loss: 0.3693 - accuracy: 0.8955 - val_loss: 0.2995 - val_accuracy: 0.8999
Epoch 3/15
355/355 [==============================] - 4s 10ms/sample - loss: 0.3097 - accuracy: 0.8955 - val_loss: 0.2867 - val_accuracy: 0.8999
Epoch 4/15
355/355 [==============================] - 3s 10ms/sample - loss: 0.2923 - accuracy: 0.8955 - val_loss: 0.2688 - val_accuracy: 0.8999
Epoch 5/15
355/355 [==============================] - 4s 10ms/sample - loss: 0.2780 - accuracy: 0.8955 - val_loss: 0.2516 - val_accuracy: 0.8999
Epoch 6/15
355/355 [==============================] - 3s 10ms/sample - loss: 0.2591 - accuracy: 0.8956 - val_loss: 0.2297 - val_accuracy: 0.8999
Epoch 7/15
355/355 [==============================] - 3s 10ms/sample - loss: 0.2380 -

In [42]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [43]:
test_pred = model.predict(X_te, verbose=1)

44/44 [==============================] - 0s 8ms/sample


In [44]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [45]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

ACC      : 98.8788%
PRECISION: 82.5893%
RECALL   : 82.5893%
F1       : 82.5893%


In [46]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

       vaso       0.88      0.95      0.92       534
    oclusao       0.53      0.80      0.64        49
 desc_placa       0.25      0.15      0.19        34
    trajeto       0.00      0.00      0.00        20
      placa       1.00      0.06      0.11        34
modificador       0.00      0.00      0.00         1

  micro avg       0.83      0.83      0.83       672
  macro avg       0.80      0.83      0.79       672



### BI-LSTM-CRF

In [47]:
import keras

Using TensorFlow backend.


In [48]:
keras.__version__

'2.2.4'

In [49]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

KeyError: 'Cannot detect if using keras or tf.keras.'

In [ ]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=32, input_length=max_len, mask_zero=True)(input)
model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(model)
# model = TimeDistributed(Dense(64, activation="relu"))(model)
crf = CRF(n_tags)
out = crf(model)

In [ ]:
model = Model(input, out)

In [ ]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=15, validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
test_pred = model.predict(X_te, verbose=1)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("ENDPAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("ACC      : {:.4%}".format(accuracy_score(test_labels, pred_labels)))
print("PRECISION: {:.4%}".format(precision_score(test_labels, pred_labels)))
print("RECALL   : {:.4%}".format(recall_score(test_labels, pred_labels)))
print("F1       : {:.4%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
print(classification_report(test_labels, pred_labels))